# Соревнования про медицину

https://inclass.kaggle.com/c/competition-2-yandex-shad-spring-2017

В файлах X.train и y.train задана обучающая выборка, X.test - выборка, для которой необходимо сделать предсказание.
Признаки

Для каждого пациента задано описание в виде 1330 признаков:

    Столбцы с V2 по V331 соответствуют различным медицинским показателям.
    Столбцы с V332 по V1331 соответствуют генетическими данными пациента.

Обратите внимание, что в случае неизвестного значения указано NaN.
Типы признаков

В файле MetaData содержится подробное описание типов каждого столбца признаков:

    Numeric - числовые признаки;
    Category - порядок значений нам не важен;
    Ordered Category - порядок значений важен;
    Генетические данные - их значения соответствуют частоте конкретного аллеля.



In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# For sklearn 0.18
# import sklearn.model_selection as skcv
import sklearn.cross_validation as skcv
import sklearn.ensemble as sk
import sklearn.preprocessing as skpp

In [3]:
X_train = pd.read_csv('cont_02/X.train.csv')
X_test = pd.read_csv('cont_02/X.test.csv')
Y_train = pd.read_csv('cont_02/y.train.csv')

## Базовое решение

In [7]:
imputer = skpp.Imputer(strategy='most_frequent')
X_train = imputer.fit_transform(X_train.values)
X_test = imputer.transform(X_test.values)

In [9]:
clf = sk.RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=1)
clf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [11]:
-np.mean(skcv.cross_val_score(clf, X_train, Y_train.label.values, scoring='log_loss', cv=5))

0.22601795516555662

In [12]:
clf.fit(X_train, Y_train.label.values)
clf.predict_proba(X_test)[:, 1]

array([ 0.   ,  0.02 ,  0.02 , ...,  0.075,  0.01 ,  0.095])